In [1]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc 
import dash_html_components as html 
import pandas as pd
import plotly.express as px
from datetime import date

from dash.dependencies import Input, Output, State, ALL, MATCH, ALLSMALLER

### HELLO WEB+DB PRESS APP

In [2]:
app = JupyterDash(__name__)

app.layout = html.H1('HELLO WEB+DB PRESS')

app.run_server(mode='inline')

### With Image App

In [3]:
app = JupyterDash(__name__)

app.layout = html.Div([
    html.P('Show Image', style={'backgroundColor': 'red', 'color': 'white'}),
    html.Img(src='assets/kyoto2.png')
])
app.run_server(mode='inline')

### グラフの表示

In [4]:
df = pd.read_csv('data/kakei_data.csv', index_col=0, parse_dates=['date'])

app = JupyterDash(__name__)

app.layout = html.Div([
    html.H1('Graphコンポーネントの利用'),
    
    dcc.Graph(
        figure=px.line(df, x='date', y='スポーツ', title='スポーツ支出')
    )
])

app.run_server(mode='inline')

### MARKDOWN

In [5]:
app = JupyterDash(__name__)

app.layout = dcc.Markdown('''
    # Markdownを使います
    ## Dash Components
    ### dash html components
    - htmlタグを提供します
    ### dash core components 
    - 様々なツールを提供します
''', style={"color": 'green', 'backgroundColor': 'black', 'padding': '2%'})

app.run_server(mode='inline')

In [6]:
from datetime import date

app = JupyterDash(__name__)

four_styles = {'margin': '3%',
                'width': '40%',
                'display': 'inline-block',
                'verticalAlign': 'top'}
selection = ['tokyo', 'osaka', 'fukuoka']

app.layout = html.Div([    
    html.Div([dcc.Dropdown(options=[{'value': i, 'label': i} for i in selection],
                value='tokyo')], style=four_styles),
    html.Div([
        dcc.Input(placeholder='文字を入力してください'),
        ], style=four_styles),
    html.Div([
        dcc.DatePickerRange(start_date=date(2017, 12, 18), end_date=date(2020, 6,22),),
        ],style=four_styles),
    html.Div([
            html.Button('HELLO PUSH ME')
        ], style=four_styles)
])

app.run_server(mode='inline')

In [7]:
app = JupyterDash(__name__)

graph_type_list = [px.line, px.scatter, px.bar]

app.layout = html.Div([
    dcc.Dropdown(
        id = 'my_dropdown',
        options = [{'label': type_.__name__, 'value': num} for num, type_ in enumerate(graph_type_list)],
        value = 0
    ),
    html.Button(
        id='my_button',
        children='Update Graph'
    ),
    dcc.Graph(
        id='my_graph',
    ),
])

@app.callback(Output('my_graph', 'figure'),
             [Input('my_button', 'n_clicks')],
            [State('my_dropdown', 'value')]
            )
def update_graph(n_clicks, selected_value):
    return graph_type_list[selected_value](x=[1,2,3,4,5], y=[1,2,3,4,5])

app.run_server(mode='inline')

In [8]:
df = pd.read_csv('data/kakei_data.csv', index_col=0, parse_dates=['date'])
df.iloc[:, :-1] = df.iloc[:, :-1].astype('float')

app = JupyterDash(__name__)

app.layout = html.Div([    
    html.Button(id='my_button', children='PUSH ME'),
    html.Div(id='my_div', children=[])
])


@app.callback(Output('my_div', 'children'),
            [Input('my_button', 'n_clicks')],
            [State('my_div', 'children')],
            prevent_initial_call=True)
def add_components(n_clicks, children):
    new_components = html.Div([
        dcc.Dropdown(id={'type': 'graph_dropdown',
                        'index': n_clicks
                        },
                    options = [{'label': c, 'value': c} for c in list(df.columns[:-1])],
                    value = [df.columns[:-1][n_clicks]],
                    multi=True
                    ),
        dcc.Graph(id={'type':'my_graph', 'index': n_clicks})
    ])
    children.append(new_components)
    return children

@app.callback(Output({'type': 'my_graph', 'index': MATCH}, 'figure'),
            [Input({'type': 'graph_dropdown', 'index': MATCH}, 'value')])
def update_graph(selected_value):
    return px.line(df, x='date', y=selected_value)

app.run_server(mode='inline')